In [1]:
import pandas as pd
import numpy as np

In [2]:
popinc = pd.read_csv("https://media.githubusercontent.com/media/MIDS-at-Duke/pds2021-opioids-team-2-ids720/data_merging/20_intermediate_files/population_2000_2020_inc.csv?token=AVKGWH4IHMMTY4IFIP4BRBTBS4OUU")
death = pd.read_csv("/Users/Aarushi/Duke/MIDS - Fall 2021/Fall 2021/720_IDS_PDS/pds2021-opioids-team-2-ids720/20_intermediate_files/vital_stata.csv")

In [3]:
# Looking at the distribution of Population and Income data
popinc.head()

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,State_Code,fips_code,Median_Income_2010,Income_Error_Margin
0,1,1,Alabama,Autauga County,2000,43671.0,AL,1001,53255.0,2420.0
1,1,3,Alabama,Baldwin County,2000,140415.0,AL,1003,50147.0,1046.0
2,1,5,Alabama,Barbour County,2000,29038.0,AL,1005,33219.0,1889.0
3,1,7,Alabama,Bibb County,2000,20826.0,AL,1007,41770.0,3139.0
4,1,9,Alabama,Blount County,2000,51024.0,AL,1009,45549.0,1804.0


In [4]:
#Population Income data types
popinc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65373 entries, 0 to 65372
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   STATE                65373 non-null  int64  
 1   COUNTY               65373 non-null  int64  
 2   STNAME               65373 non-null  object 
 3   CTYNAME              65373 non-null  object 
 4   Year                 65373 non-null  int64  
 5   Population           65339 non-null  float64
 6   State_Code           65373 non-null  object 
 7   fips_code            65373 non-null  int64  
 8   Median_Income_2010   64512 non-null  float64
 9   Income_Error_Margin  64512 non-null  float64
dtypes: float64(3), int64(4), object(3)
memory usage: 5.0+ MB


In [5]:
death.head()

,Unnamed: 0,County,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State_Code,County_Name
0,1,"Baldwin County, AL",1003.0,2003,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,AL,Baldwin County
1,38,"Jefferson County, AL",1073.0,2003,Drug poisonings (overdose) Unintentional (X40-...,D1,37.0,AL,Jefferson County
2,39,"Jefferson County, AL",1073.0,2003,All other drug-induced causes,D9,32.0,AL,Jefferson County
3,54,"Mobile County, AL",1097.0,2003,Drug poisonings (overdose) Unintentional (X40-...,D1,26.0,AL,Mobile County
4,106,"Cochise County, AZ",4003.0,2003,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,AZ,Cochise County


In [6]:
print(death.Deaths.min())
print(death.Deaths.max())

10
Missing


In [7]:
#Vital Statistics data types
death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10383 entries, 0 to 10382
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       10383 non-null  int64  
 1   County                           10383 non-null  object 
 2   County Code                      10383 non-null  float64
 3   Year                             10383 non-null  int64  
 4   Drug/Alcohol Induced Cause       10383 non-null  object 
 5   Drug/Alcohol Induced Cause Code  10383 non-null  object 
 6   Deaths                           10383 non-null  object 
 7   State_Code                       10383 non-null  object 
 8   County_Name                      10383 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 730.2+ KB


In [8]:
death["Drug/Alcohol Induced Cause"].unique()

array(['Drug poisonings (overdose) Unintentional (X40-X44)',
       'All other drug-induced causes',
       'Drug poisonings (overdose) Suicide (X60-X64)',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'Drug poisonings (overdose) Homicide (X85)'], dtype=object)

In [9]:
death = death.replace("Missing", np.NaN)

In [10]:
death["Deaths"] = death["Deaths"].astype(float)

In [11]:
# Collapsing the dataset to get total number of deaths per county per year
mortality = death.groupby(["Year","State_Code","County_Name"], as_index=False)['Deaths'].apply(lambda x: x.sum())
mortality.head()

,Year,State_Code,County_Name,Deaths
0,2003,AL,Baldwin County,10.0
1,2003,AL,Jefferson County,69.0
2,2003,AL,Mobile County,26.0
3,2003,AR,Pulaski County,18.0
4,2003,AZ,Cochise County,11.0


In [12]:
# Changing columns name for merging
popinc = popinc.rename({"CTYNAME":"County_Name"}, axis =1)
popinc

,STATE,COUNTY,STNAME,County_Name,Year,Population,State_Code,fips_code,Median_Income_2010,Income_Error_Margin
0,1,1,Alabama,Autauga County,2000,43671.0,AL,1001,53255.0,2420.0
1,1,3,Alabama,Baldwin County,2000,140415.0,AL,1003,50147.0,1046.0
2,1,5,Alabama,Barbour County,2000,29038.0,AL,1005,33219.0,1889.0
3,1,7,Alabama,Bibb County,2000,20826.0,AL,1007,41770.0,3139.0
4,1,9,Alabama,Blount County,2000,51024.0,AL,1009,45549.0,1804.0
...,...,...,...,...,...,...,...,...,...,...
65368,56,37,Wyoming,Sweetwater County,2020,42673.0,WY,56037,69828.0,1963.0
65369,56,39,Wyoming,Teton County,2020,23497.0,WY,56039,70271.0,7421.0
65370,56,41,Wyoming,Uinta County,2020,20215.0,WY,56041,58346.0,4424.0
65371,56,43,Wyoming,Washakie County,2020,7760.0,WY,56043,48379.0,4970.0


In [13]:
# Merging Population with Deaths
merged = pd.merge(
    mortality[
        [
            "Year",
            "State_Code",
            "County_Name",
            "Deaths"
        ]
    ],
    popinc[
        [
            "County_Name",
            "Year",
            "State_Code",
            "Population",
            "Median_Income_2010",
            "Income_Error_Margin",
        ]
    ],
    how="outer",
    on=["Year", "State_Code", "County_Name"],
    validate="m:1",
    indicator=True,
)


In [14]:
#Viewing the merged dataset
merged.head()

,Year,State_Code,County_Name,Deaths,Population,Median_Income_2010,Income_Error_Margin,_merge
0,2003,AL,Baldwin County,10.0,151707.0,50147.0,1046.0,both
1,2003,AL,Jefferson County,69.0,660486.0,45244.0,641.0,both
2,2003,AL,Mobile County,26.0,397707.0,40996.0,723.0,both
3,2003,AR,Pulaski County,18.0,366375.0,45121.0,783.0,both
4,2003,AZ,Cochise County,11.0,121175.0,44876.0,1551.0,both


In [15]:
merged.loc[merged["Population"].isna()]["State_Code"].unique()

array(['DC', 'NM', 'IN', 'PA', 'VA', 'CO', 'LA', 'SD'], dtype=object)

In [16]:
merged.loc[merged["Deaths"].isna()]["State_Code"].unique()

array(['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID',
       'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN',
       'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
       'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
       'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [17]:
[merged["Deaths"].isna().sum()]

[57520]

In [18]:
# Verifying whether the data merged correctly
check = merged[merged["_merge"] != "both"]
check
# There are cases where data was only available for each respective dataset

,Year,State_Code,County_Name,Deaths,Population,Median_Income_2010,Income_Error_Margin,_merge
59,2003,DC,District of Columbia,106.0,NaN,NaN,NaN,left_only
251,2003,NM,Dona Ana County,19.0,NaN,NaN,NaN,left_only
468,2004,DC,District of Columbia,102.0,NaN,NaN,NaN,left_only
676,2004,NM,Dona Ana County,17.0,NaN,NaN,NaN,left_only
925,2005,DC,District of Columbia,87.0,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
65405,2020,WY,Sweetwater County,NaN,42673.0,69828.0,1963.0,right_only
65406,2020,WY,Teton County,NaN,23497.0,70271.0,7421.0,right_only
65407,2020,WY,Uinta County,NaN,20215.0,58346.0,4424.0,right_only
65408,2020,WY,Washakie County,NaN,7760.0,48379.0,4970.0,right_only


In [19]:
check_right = merged[merged["_merge"] == "right_only"]
check_right

,Year,State_Code,County_Name,Deaths,Population,Median_Income_2010,Income_Error_Margin,_merge
7890,2000,AL,Autauga County,NaN,43671.0,53255.0,2420.0,right_only
7891,2000,AL,Baldwin County,NaN,140415.0,50147.0,1046.0,right_only
7892,2000,AL,Barbour County,NaN,29038.0,33219.0,1889.0,right_only
7893,2000,AL,Bibb County,NaN,20826.0,41770.0,3139.0,right_only
7894,2000,AL,Blount County,NaN,51024.0,45549.0,1804.0,right_only
...,...,...,...,...,...,...,...,...
65405,2020,WY,Sweetwater County,NaN,42673.0,69828.0,1963.0,right_only
65406,2020,WY,Teton County,NaN,23497.0,70271.0,7421.0,right_only
65407,2020,WY,Uinta County,NaN,20215.0,58346.0,4424.0,right_only
65408,2020,WY,Washakie County,NaN,7760.0,48379.0,4970.0,right_only


In [20]:
check[check["_merge"] == "left_only"]

,Year,State_Code,County_Name,Deaths,Population,Median_Income_2010,Income_Error_Margin,_merge
59,2003,DC,District of Columbia,106.0,NaN,NaN,NaN,left_only
251,2003,NM,Dona Ana County,19.0,NaN,NaN,NaN,left_only
468,2004,DC,District of Columbia,102.0,NaN,NaN,NaN,left_only
676,2004,NM,Dona Ana County,17.0,NaN,NaN,NaN,left_only
925,2005,DC,District of Columbia,87.0,NaN,NaN,NaN,left_only
1157,2005,NM,Dona Ana County,18.0,NaN,NaN,NaN,left_only
1408,2006,DC,District of Columbia,113.0,NaN,NaN,NaN,left_only
1488,2006,IN,La Porte County,10.0,NaN,NaN,NaN,left_only
1656,2006,NM,Dona Ana County,26.0,NaN,NaN,NaN,left_only
1944,2007,DC,District of Columbia,90.0,NaN,NaN,NaN,left_only


In [21]:
#Imputing death values where death data is missing
merged["Deaths"] = merged["Deaths"].apply(lambda x: x if not np.isnan(x) else np.random.randint(0, 10))

In [22]:
[merged["Deaths"].isna().sum()]

[0]

In [23]:
#Checking the counties where we did not have population and income data
check[check["_merge"]=="left_only"].County_Name.value_counts()

District of Columbia    13
Dona Ana County         13
La Porte County          9
Mc Kean County           1
Clifton Forge city       1
Name: County_Name, dtype: int64

In [24]:
#Checking how many values by state are unavailable
check[check["_merge"]=="left_only"].State_Code.value_counts()

DC    13
NM    13
IN     9
PA     1
VA     1
Name: State_Code, dtype: int64

In [25]:
#Checking unavailable values based on year
check[check["_merge"]=="left_only"].Year.value_counts().sort_values()

2003    2
2004    2
2005    2
2008    2
2006    3
2007    3
2009    3
2010    3
2011    3
2012    3
2013    3
2014    3
2015    5
Name: Year, dtype: int64

For now we will leave these values as it is, and proceed with calculating deaths per capita to select the states, for our analysis. If the per capita rate for any of these states is close to our Analysis states (TX,FL, WA) we can think of how to impute these values

In [26]:
# Checking rows where number of deaths is not available
merged[merged["Deaths"]==np.NaN]

,Year,State_Code,County_Name,Deaths,Population,Median_Income_2010,Income_Error_Margin,_merge


In [27]:
#Checking if there are any missing values in the merged dataset for population
#merged.loc[merged["Population"].isna()]
# Here we see that most of the values are similar to the issue in the left merge column implying that we do not have population and income data for those specific counties 
#However we have 4 rows where the population datat is missing in the population file as well.
#On further review, it is for BEdford City, VA and the poppulation estimate as per google is ~7000 which is pretty irrelevant imo

In [28]:
# Calculating county wise death per capita
# Replacing the "Missing" value in Death column by Nan
#merged = merged.replace("Missing", np.NaN)
#merged.info() #Checking the datatype of Deaths column


In [29]:
merged["Death_per_cap"] = (merged["Deaths"] / merged ["Population"])

In [30]:
merged.columns

Index(['Year', 'State_Code', 'County_Name', 'Deaths', 'Population',
       'Median_Income_2010', 'Income_Error_Margin', '_merge', 'Death_per_cap'],
      dtype='object')

In [31]:
#Reordering the columns 
merged = merged[
    [
        "Year",
        "State_Code",
        "County_Name",
        "Population",
        "Deaths",
        "Death_per_cap",
        "Median_Income_2010",
        "Income_Error_Margin",
        "_merge",

    ]
]


In [32]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65410 entries, 0 to 65409
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Year                 65410 non-null  int64   
 1   State_Code           65410 non-null  object  
 2   County_Name          65410 non-null  object  
 3   Population           65339 non-null  float64 
 4   Deaths               65410 non-null  float64 
 5   Death_per_cap        65339 non-null  float64 
 6   Median_Income_2010   64512 non-null  float64 
 7   Income_Error_Margin  64512 non-null  float64 
 8   _merge               65410 non-null  category
dtypes: category(1), float64(5), int64(1), object(2)
memory usage: 4.6+ MB


In [33]:
# Looking at the death per cap values to shortlist the control states for our analysis
merged.groupby(by=(["Year","State_Code"])).mean().sort_values(by="Death_per_cap")

Population      Deaths  Death_per_cap  Median_Income_2010  \
Year State_Code                                                                 
2017 DE          319314.000000    2.333333       0.000009        55567.666667   
2018 CT          446820.125000    3.625000       0.000011        68935.125000   
2020 MA          492398.142857    4.214286       0.000012        62828.642857   
     CT          444625.750000    3.125000       0.000013        68935.125000   
     NJ          422970.047619    3.571429       0.000013        70629.904762   
...                        ...         ...            ...                 ...   
2011 DC                    NaN   79.000000            NaN                 NaN   
2012 DC                    NaN   71.000000            NaN                 NaN   
2013 DC                    NaN  112.000000            NaN                 NaN   
2014 DC                    NaN   90.000000            NaN                 NaN   
2015 DC                    NaN  114.000000            NaN                 NaN   

                 Income_Error_Margin  
Year State_Code                       
2017 DE                  1041.000000  
2018 CT                  1234.125000  
2020 MA                  1992.142857  
     CT                  1234.125000  
     NJ                  1289.095238  
...                              ...  
2011 DC                          NaN  
2012 DC                          NaN  
2013 DC                          NaN  
2014 DC                          NaN  
2015 DC                          NaN  

[1042 rows x 5 columns]

In [34]:
merged.duplicated(["Year","State_Code","County_Name"]).sum()

0

In [35]:
# For WA - IL, ID, WI
# For FL - IL, WA, ID
# For TX - UT, CO, KS

In [36]:
print(merged.Year.min())
print(merged.Year.max())

2000
2020


In [37]:
merged.Year.unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2000, 2001, 2002, 2016, 2017, 2018, 2019, 2020])

In [38]:
years = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015]
merged_final = merged[merged["Year"].isin(years)]

In [39]:
print(merged_final.Year.min())
print(merged_final.Year.max())

2003
2015


In [40]:
merged_final.to_csv('/Users/Aarushi/Duke/MIDS - Fall 2021/Fall 2021/720_IDS_PDS/pds2021-opioids-team-2-ids720/20_intermediate_files/mortality_merged.csv', encoding='utf-8', index=False)
